<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250429(%E7%B5%90%E6%A7%8B%E5%8C%96%E8%AA%BF%E6%95%B4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import urllib.request as req
import bs4 as bs
import pandas as pd

# pandas
table = []

dn = "pchome/"
if not os.path.exists(dn):
    os.makedirs(dn)

url = "https://24h.pchome.com.tw/search/?q=%E5%90%B9%E9%A2%A8%E6%A9%9F"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())
links = html.find_all("a", {"class":"c-prodInfoV2__link"})
for link in links:
    link_href = "https://24h.pchome.com.tw" + link["href"]
    print(link_href)
    # enter
    p_resp = req.urlopen(link_href)
    p_html = bs.BeautifulSoup(p_resp.read())
    # 產品品牌
    p_brand = p_html.find("span", {"class":"o-prodMainName__colorSecondary"})
    p_brand_text = p_brand.text.strip()
    # 產品名稱
    p_name = p_html.find("h1", {"class":"o-prodMainName__grayDarkest"})
    p_name_text = p_name.text.strip()
    # 滿額贈
    p_labels = p_html.find_all("div", {"class":"c-label__rectangle"})
    p_full_prize = "N"
    for p_label in p_labels:
        if "滿額贈" in p_label.text:
            p_full_prize = "Y"
            break
    # 評分
    p_rating_all = p_html.find("div", {"class":"c-ratingIcon__flex"})
    if not p_rating_all == None:
        p_rating = p_rating_all.find("div", {"class":"c-ratingIcon__textNumber"})
        p_rating_count = p_rating_all.find("a")
        p_rating_text = p_rating.text.strip()
        p_rating_count_text = p_rating_count.text.strip().replace("則評價", "")
    else:
        p_rating_text = "-"
        p_rating_count_text = "-"

    # 原價/價格
    p_pricebox = p_html.find("div", {"class":"o-prodPrice__priceBox"})
    p_price_now = p_pricebox.find("div", {"class":"o-prodPrice__price"})
    p_price_now_text = p_price_now.text.strip()
    p_price_origin = p_pricebox.find("div", {"class":"o-prodPrice__originalPrice"})
    if p_price_origin == None:
        p_price_origin_text = "-"
    else:
        p_price_origin_text = p_price_origin.text.strip()
    # 介紹
    p_intro_list = p_html.find_all("li", {"class":"c-blockCombine__item--prodSlogan"})
    p_intro_text_list = []
    for p_intro in p_intro_list:
        p_intro_text = p_intro.text.strip()
        p_intro_text_list.append(p_intro_text)
    p_intro_text_join = ""
    for p_intro_text in p_intro_text_list:
        p_intro_text_join = p_intro_text_join + "- " + p_intro_text + "\n"
    # 大圖片
    p_img = p_html.find("div", {"class":"c-radiusPhotoImage__img"}).find("img")
    p_img_src = p_img["src"]
    if not "https:" in p_img_src:
        p_img_src = "https:" + p_img_src
    # 幫你存
    fn = link_href.split("/")[-1]
    fn = dn + fn + ".jpg"
    req.urlretrieve(p_img_src, fn)
    # print("原價:", p_price_origin_text)
    # print("目前售價:", p_price_now_text)
    # print("評分:", p_rating_text)
    # print("幾則:", p_rating_count_text)
    # print("品牌:", p_brand_text)
    # print("商品:", p_name_text)
    # print("滿額贈:", p_full_prize)
    # print("介紹:", p_intro_text_join)
    # print("圖片位址:", p_img_src)
    data = {
        "原價":p_price_origin_text,
        "現價":p_price_now_text,
        "評分":p_rating_text,
        "評價個數":p_rating_count_text,
        "品牌":p_brand_text,
        "名稱":p_name_text,
        "滿額贈":p_full_prize,
        "介紹":p_intro_text_join,
        "圖片網址":p_img_src,
        "圖片儲存":fn,
    }
    table.append(data)
df = pd.json_normalize(table)
df.to_csv("pchome.csv", encoding="utf-8")
df

In [ ]:
# 名字
# 原價/目前售價
# 滿額贈是否有(Y/N)
# 進去以後
# 評價幾分/幾則
# 介紹
# 圖片的網址

In [ ]:
import requests
import json
url = "https://www.thsrc.com.tw/TimeTable/Search"
payload_s = """SearchType: S
Lang: TW
StartStation: TaiPei
EndStation: ZuoYing
OutWardSearchDate: 2025/04/28
OutWardSearchTime: 15:30
ReturnSearchDate: 2025/04/28
ReturnSearchTime: 15:30
DiscountType: """
payload = {}
for spl in payload_s.split("\n"):
    k, v = spl.split(": ")
    k, v = k.strip(), v.strip()
    payload[k] = v

resp = requests.post(url, payload)
resp_json = json.loads(resp.text)
data = resp_json["data"]
trains = data["DepartureTable"]["TrainItem"]
pd.json_normalize(trains)

In [ ]:
def get_prod_info(link_href):
    # enter
    p_resp = req.urlopen(link_href)
    p_html = bs.BeautifulSoup(p_resp.read())
    # 產品品牌
    p_brand = p_html.find("span", {"class":"o-prodMainName__colorSecondary"})
    p_brand_text = p_brand.text.strip()
    # 產品名稱
    p_name = p_html.find("h1", {"class":"o-prodMainName__grayDarkest"})
    p_name_text = p_name.text.strip()
    # 滿額贈
    p_labels = p_html.find_all("div", {"class":"c-label__rectangle"})
    p_full_prize = "N"
    for p_label in p_labels:
        if "滿額贈" in p_label.text:
            p_full_prize = "Y"
            break
    # 評分
    p_rating_all = p_html.find("div", {"class":"c-ratingIcon__flex"})
    if not p_rating_all == None:
        p_rating = p_rating_all.find("div", {"class":"c-ratingIcon__textNumber"})
        p_rating_count = p_rating_all.find("a")
        p_rating_text = p_rating.text.strip()
        p_rating_count_text = p_rating_count.text.strip().replace("則評價", "")
    else:
        p_rating_text = "-"
        p_rating_count_text = "-"

    # 原價/價格
    p_pricebox = p_html.find("div", {"class":"o-prodPrice__priceBox"})
    p_price_now = p_pricebox.find("div", {"class":"o-prodPrice__price"})
    p_price_now_text = p_price_now.text.strip()
    p_price_origin = p_pricebox.find("div", {"class":"o-prodPrice__originalPrice"})
    if p_price_origin == None:
        p_price_origin_text = "-"
    else:
        p_price_origin_text = p_price_origin.text.strip()
    # 介紹
    p_intro_list = p_html.find_all("li", {"class":"c-blockCombine__item--prodSlogan"})
    p_intro_text_list = []
    for p_intro in p_intro_list:
        p_intro_text = p_intro.text.strip()
        p_intro_text_list.append(p_intro_text)
    p_intro_text_join = ""
    for p_intro_text in p_intro_text_list:
        p_intro_text_join = p_intro_text_join + "- " + p_intro_text + "\n"
    # 大圖片
    p_img = p_html.find("div", {"class":"c-radiusPhotoImage__img"}).find("img")
    p_img_src = p_img["src"]
    if not "https:" in p_img_src:
        p_img_src = "https:" + p_img_src
    # 幫你存
    dn = "pchome/"
    if not os.path.exists(dn):
        os.makedirs(dn)

    fn = link_href.split("/")[-1]
    fn = dn + fn + ".jpg"
    req.urlretrieve(p_img_src, fn)

    data = {
        "原價":p_price_origin_text,
        "現價":p_price_now_text,
        "評分":p_rating_text,
        "評價個數":p_rating_count_text,
        "品牌":p_brand_text,
        "名稱":p_name_text,
        "滿額贈":p_full_prize,
        "介紹":p_intro_text_join,
        "圖片網址":p_img_src,
        "圖片儲存":fn,
    }
    return data

def get_page(url):
    table = []

    resp = req.urlopen(url)
    html = bs.BeautifulSoup(resp.read())
    links = html.find_all("a", {"class":"c-prodInfoV2__link"})
    for link in links:
        link_href = "https://24h.pchome.com.tw" + link["href"]
        data = get_prod_info(link_href)
        table.append(data)
    return table

In [ ]:
import os
import urllib.request as req
import bs4 as bs
import pandas as pd


total = []
for i in range(1):
    page = i + 1
    url = "https://24h.pchome.com.tw/search/?q=%E5%90%B9%E9%A2%A8%E6%A9%9F&p={}".format(page)
    total = total + get_page(url)

pd.json_normalize(total)